In [ ]:
%matplotlib inline
import numpy as np
import numpy
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from rayleigh_diagnostics import AZ_Avgs
import pylab 

ek_dir = './'

In [ ]:
def gen_s_Z_grid(radius,costheta, sintheta):
    n_r = len(radius)
    n_theta =len(az.costheta)        

    rad = radius
    rmax = numpy.max(radius)
    s_sph = pylab.dot(sintheta.reshape(n_theta,1),rad.reshape(1,n_r))  # s,z values at each
    z_sph = pylab.dot(costheta.reshape(n_theta,1),rad.reshape(1,n_r))  # r, theta location

    s1d = s_sph.reshape(n_theta*n_r)
    z1d = z_sph.reshape(n_theta*n_r)

    nx = n_r  # Could make this anything -- n_r seems sensible place to start
    s = np.linspace(0,rmax,nx)
    z =  np.linspace(-rmax,rmax,2*nx)
    s2d, z2d = np.meshgrid(s,z)  
    
    return s1d, z1d, s2d, z2d

def az_to_cyl(val, s1d, z1d, s2d, z2d):
    ndim = val.shape[0]*val.shape[1]
    val1d = val.reshape(ndim)
    val_interp = griddata((s1d, z1d), val1d, (s2d, z2d), method='cubic', fill_value=0)
    return val_interp

def az_to_cyl2(az,qind,tind, s1d, z1d, s2d, z2d, remove_mean=False):

    temp = az.vals[:,:,az.lut[qind],tind]
    
    
    n1,n2 = temp.shape
    if (remove_mean):
        for i in range(n2):
            temp[:,i] = temp[:,i]-numpy.mean(temp[:,i])
    temp[0:3,:]=0
    temp[n1-3:n1,:] = 0
    temp[:,n2-3:n2] = 0
    temp[:,0:3] = 0

    print(n1,n2)
    ndim = temp.shape[0]*temp.shape[1]
    temp1d = temp.reshape(ndim)
    val_interp = griddata((s1d, z1d), temp1d, (s2d, z2d), method='linear',fill_value=0)
    return val_interp

def cyl_sph(q_r,q_t,costheta,sintheta):
    #Assume costheta and sintheta are on the cylindrical grid
    #e.g. costheta(s,z) = cosin(theta) at location s, z
    
    b_s = q_r*sintheta + q_t*costheta
    b_z = q_r*costheta - q_t*sintheta
    
    return b_s, b_z


In [ ]:
az_file = '00210100'

az = AZ_Avgs(az_file,path=ek_dir + 'AZ_Avgs/')
print(az.lut[1263])

radius = az.radius
costheta = az.costheta
sintheta = az.sintheta

s1d, z1d, s2d, z2d = gen_s_Z_grid(radius,costheta, sintheta)


ugradu_r= az_to_cyl2(     az,1201,0, s1d,z1d,s2d,z2d)
ugradu_t= az_to_cyl2(     az,1202,0, s1d,z1d,s2d,z2d)
coriolis_r = az_to_cyl2(  az,1219,0, s1d,z1d,s2d,z2d)
coriolis_t= az_to_cyl2(   az,1220,0, s1d,z1d,s2d,z2d)
viscous_r= az_to_cyl2(    az,1228,0, s1d,z1d,s2d,z2d)
viscous_t= az_to_cyl2(    az,1229,0, s1d,z1d,s2d,z2d)
cen_r = az_to_cyl2(    az,1263,0, s1d,z1d,s2d,z2d)
cen_t = az_to_cyl2(    az,1264,0, s1d,z1d,s2d,z2d)

temp_cyl = az_to_cyl2( az, 501, 0, s1d, z1d, s2d, z2d, remove_mean=True)
vphi_cyl = az_to_cyl2(    az,3,0, s1d,z1d,s2d,z2d)

pressure_r1 = az_to_cyl2( az,1237,0, s1d,z1d,s2d,z2d)
pressure_t1 = az_to_cyl2( az,1238,0, s1d,z1d,s2d,z2d)

pressure_r2 = az_to_cyl2( az,1237,1, s1d,z1d,s2d,z2d)
pressure_t2 = az_to_cyl2( az,1238,1, s1d,z1d,s2d,z2d)

pressure_r = 0.5*(pressure_r1 + pressure_r2)
pressure_t = 0.5*(pressure_t1 + pressure_t2)

R = numpy.sqrt(s2d*s2d+z2d*z2d)

cos2d = 0*R
sin2d = 0*R


cos2d = z2d/R
sin2d = s2d/R


print(np.shape(ugradu_r))
print(np.shape(cos2d))
cos2d.shape

plt.figure(1)
plt.imshow(temp_cyl)
plt.show()
#print(pressure_r[64,:])

In [ ]:
import numpy
figdpi=300
sizetuple=(10,5)
ugradu_s, ugradu_z = cyl_sph(ugradu_r,ugradu_t,cos2d,sin2d)
coriolis_s, coriolis_z = cyl_sph(coriolis_r,coriolis_t,cos2d,sin2d)
viscous_s, viscous_z = cyl_sph(viscous_r,viscous_t,cos2d,sin2d)
pressure_s, pressure_z = cyl_sph(pressure_r,pressure_t,cos2d,sin2d)
centri_s, centri_z = cyl_sph(cen_r,cen_t,cos2d,sin2d)
n_r = len(radius)
z = z2d[:,0]
s = s2d[0,:]

s_index = 50

fig, ax = plt.subplots(ncols=2,figsize=sizetuple,dpi=figdpi)

s_tot = -ugradu_s + coriolis_s + viscous_s + pressure_s  + centri_s
ax[0].plot(z,-ugradu_s[:,s_index],label='ugradu_s')
ax[0].plot(z,coriolis_s[:,s_index],label='coriolis_s')
ax[0].plot(z,viscous_s[:,s_index],label='viscous_s')
ax[0].plot(z,pressure_s[:,s_index],label='pressure_s')
ax[0].plot(z,centri_s[:,s_index],label='centri_s')
ax[0].plot(z,s_tot[:,s_index],color='black')
ax[0].set_title('Radial force balance')
ax[0].set_xlabel('Z')
#ax[0].set_xlim(-1,1)

z_tot = -ugradu_z + coriolis_z + viscous_z + pressure_z  + centri_z
ax[1].plot(z,-ugradu_z[:,s_index],label='ugradu_z')
ax[1].plot(z,coriolis_z[:,s_index],label='coriolis_z')
ax[1].plot(z,viscous_z[:,s_index],label='viscous_z')
ax[1].plot(z,pressure_z[:,s_index],label='pressure_z')
ax[1].plot(z,centri_z[:,s_index],label='centri_z')
ax[1].plot(z,z_tot[:,s_index],color='black')
ax[1].set_title('Z-force balance')
ax[1].set_xlabel('Z')

ax[0].legend(loc='best', shadow=True)
ax[1].legend(loc='best', shadow=True)
print(s[s_index])

plt.tight_layout()
plt.show()

print(s[s_index])

In [ ]:
import numpy
figdpi=300
sizetuple=(10,5)
ugradu_s, ugradu_z = cyl_sph(ugradu_r,ugradu_t,cos2d,sin2d)
coriolis_s, coriolis_z = cyl_sph(coriolis_r,coriolis_t,cos2d,sin2d)
viscous_s, viscous_z = cyl_sph(viscous_r,viscous_t,cos2d,sin2d)
pressure_s, pressure_z = cyl_sph(pressure_r,pressure_t,cos2d,sin2d)
centri_s, centri_z = cyl_sph(cen_r,cen_t,cos2d,sin2d)
n_r = len(radius)
z = z2d[:,0]
s = s2d[0,:]

s_index = 5

fig, ax = plt.subplots(ncols=2,figsize=sizetuple,dpi=figdpi)

s_tot = -ugradu_s + coriolis_s + viscous_s + pressure_s  + centri_s
ax[0].plot(z,-ugradu_s[:,s_index],label='ugradu_s')
ax[0].plot(z,coriolis_s[:,s_index],label='coriolis_s')
ax[0].plot(z,viscous_s[:,s_index],label='viscous_s')
ax[0].plot(z,pressure_s[:,s_index],label='pressure_s')
ax[0].plot(z,centri_s[:,s_index],label='centri_s')
ax[0].plot(z,s_tot[:,s_index],color='black')
ax[0].set_title('Radial force balance')
ax[0].set_xlabel('Z')
#ax[0].set_xlim(-1,1)

z_tot = -ugradu_z + coriolis_z + viscous_z + pressure_z  + centri_z
ax[1].plot(z,-ugradu_z[:,s_index],label='ugradu_z')
ax[1].plot(z,coriolis_z[:,s_index],label='coriolis_z')
ax[1].plot(z,viscous_z[:,s_index],label='viscous_z')
ax[1].plot(z,pressure_z[:,s_index],label='pressure_z')
ax[1].plot(z,centri_z[:,s_index],label='centri_z')
ax[1].plot(z,z_tot[:,s_index],color='black')
ax[1].set_title('Z-force balance')
ax[1].set_xlabel('Z')

ax[0].legend(loc='best', shadow=True)
ax[1].legend(loc='best', shadow=True)
print(s[s_index])

plt.tight_layout()
plt.show()

print(s[s_index])

In [ ]:
z_index = 105

fig, ax = plt.subplots(ncols=2,figsize=sizetuple,dpi=figdpi)

s_tot = -ugradu_s + coriolis_s + viscous_s + pressure_s  + centri_s
ax[0].plot(s,-ugradu_s[z_index,:],label='ugradu_s')
ax[0].plot(s,coriolis_s[z_index,:],label='coriolis_s')
ax[0].plot(s,viscous_s[z_index,:],label='viscous_s')
ax[0].plot(s,pressure_s[z_index,:],label='pressure_s')
ax[0].plot(s,centri_s[z_index,:],label='centri_s')
ax[0].plot(s,s_tot[z_index,:],color='black')
ax[0].set_title('Radial force balance')
ax[0].set_xlabel('S')
#ax[0].set_xlim(-1,1)

z_tot = -ugradu_z + coriolis_z + viscous_z + pressure_z  + centri_z
ax[1].plot(s,-ugradu_z[z_index,:],label='ugradu_z')
ax[1].plot(s,coriolis_z[z_index,:],label='coriolis_z')
ax[1].plot(s,viscous_z[z_index,:],label='viscous_z')
ax[1].plot(s,pressure_z[z_index,:],label='pressure_z')
ax[1].plot(s,centri_z[z_index,:],label='centri_z')
ax[1].plot(s,z_tot[z_index,:],color='black')
ax[1].set_title('Z-force balance')
ax[1].set_xlabel('S')

ax[0].legend(loc='best', shadow=True)
ax[1].legend(loc='best', shadow=True)


plt.tight_layout()
plt.show()

print(z[z_index])

In [ ]:
z_index = 64

fig, ax = plt.subplots(ncols=2,figsize=sizetuple,dpi=figdpi)

s_tot = -ugradu_s + coriolis_s + viscous_s + pressure_s  + centri_s
ax[0].plot(s,-ugradu_s[z_index,:],label='ugradu_s')
ax[0].plot(s,coriolis_s[z_index,:],label='coriolis_s')
ax[0].plot(s,viscous_s[z_index,:],label='viscous_s')
ax[0].plot(s,pressure_s[z_index,:],label='pressure_s')
ax[0].plot(s,centri_s[z_index,:],label='centri_s')
ax[0].plot(s,s_tot[z_index,:],color='black')
ax[0].set_title('Radial force balance')
ax[0].set_xlabel('S')
#ax[0].set_xlim(-1,1)

z_tot = -ugradu_z + coriolis_z + viscous_z + pressure_z  + centri_z
ax[1].plot(s,-ugradu_z[z_index,:],label='ugradu_z')
ax[1].plot(s,coriolis_z[z_index,:],label='coriolis_z')
ax[1].plot(s,viscous_z[z_index,:],label='viscous_z')
ax[1].plot(s,pressure_z[z_index,:],label='pressure_z')
ax[1].plot(s,centri_z[z_index,:],label='centri_z')
ax[1].plot(s,z_tot[z_index,:],color='black')
ax[1].set_title('Z-force balance')
ax[1].set_xlabel('S')

ax[0].legend(loc='best', shadow=True)
ax[1].legend(loc='best', shadow=True)


plt.tight_layout()
plt.show()

print(z[z_index])